<a href="https://colab.research.google.com/github/chernoskutov1989dv/Self-Education/blob/main/15_2_rnn_embedding_se_21_version_12ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding  # Embedding отсекает ненужные связи(нули), оптимизируя обьем памяти
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [ ]:
open ("/content/train_data_true.txt")

<_io.TextIOWrapper name='/content/train_data_true.txt' mode='r' encoding='UTF-8'>

In [ ]:

with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [ ]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

dist = list(tokenizer.word_counts.items())
print(dist[:10])


[('думайте', 1), ('позитивно', 4), ('и', 29), ('верьте', 3), ('в', 23), ('свою', 3), ('способность', 1), ('достигать', 1), ('отличных', 1), ('результатов', 1)]


In [ ]:
data = tokenizer.texts_to_sequences([texts])
#res = to_categorical(data[0], num_classes=maxWordsCount)
#print(res.shape)

res = np.array( data[0] )

inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words] for i in range(n)])  # теперь это набор чисел!!
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)


In [ ]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))  # Embedding слой должен быть первым!! 
#model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))  # 1ый рекуррентный слой
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 3, 256)            256000    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 128)               49280     
                                                                 
 dense_2 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 434,280
Trainable params: 434,280
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
31/31 [==============================] - 1s 10ms/step - loss: 6.8870 - accuracy: 0.0284
Epoch 2/50
31/31 [==============================] - 0s 10ms/step - loss: 6.2680 - accuracy: 0.0436
Epoch 3/50
31/31 [==============================] - 0s 10ms/step - loss: 5.7324 - accuracy: 0.0710
Epoch 4/50
31/31 [==============================] - 0s 10ms/step - loss: 5.4162 - accuracy: 0.0842
Epoch 5/50
31/31 [==============================] - 0s 10ms/step - loss: 5.0169 - accuracy: 0.1268
Epoch 6/50
31/31 [==============================] - 0s 10ms/step - loss: 4.4792 - accuracy: 0.1724
Epoch 7/50
31/31 [==============================] - 0s 10ms/step - loss: 3.8888 - accuracy: 0.2941
Epoch 8/50
31/31 [==============================] - 0s 10ms/step - loss: 3.3282 - accuracy: 0.4331
Epoch 9/50
31/31 [==============================] - 0s 10ms/step - loss: 2.7973 - accuracy: 0.5751
Epoch 10/50
31/31 [==============================] - 0s 10ms/step - loss: 2.3337 - accuracy: 0.6663
Epoch 11/

In [ ]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words] # теперь берем конкретные индексы, а не ОНЕ векторы!!
        inp = np.expand_dims(x, axis =0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res


res = buildPhrase("Не просто учись")
print(res)



1/1 [==============================] - 0s 20ms/step
Не просто учись а приобретай опыт не читай а впитывай не просто меняйся а трансформируйся не просто связывай а защищай не обещай а
